In [ ]:
%pip install ms-fabric-cli --quiet

# First Time Setup
If it´s the first time running the script, set the parameter 'FIRST_RUN' as 'True', else set it up as 'False'

In [ ]:
FIRST_RUN = True

# Variables

Set the following variables for the initial setup or update of the environment

In [ ]:
# Key Vault URI
key_vault_uri = f""

# Key Vault secret name with the tenant id
key_vault_tenant_id = f"tenant-id"

# Key vault secret name with the App Id of the Service Principal
key_vault_client_id = f"fabric-admin-api-sp-id"

# Key vault secret name with the secret of the Service Principal
key_vault_client_secret = f"fabric-admin-api-sp-secret"

# Id of the Security Group with the Service Principal used for the Admin API´s and Gateway Administration
group_id = f""

# Capacity ID of the capacity you want to setup. You can change it manually later in the Eventstream, 
# or add more capacities
capacity_id = f""

Don´t change the following variables unless specified by the repo owners

In [ ]:
##### DO NET CHANGE UNLESS SPECIFIED OTHERWISE ####
repo_owner = "ecotte"
repo_name = "Fabric-Monitoring-RTI"
branch = "Capacity"
folder_prefix = ""
###################################################

# Load Libraries

In [ ]:
import subprocess
import os
import json
from zipfile import ZipFile 
import shutil
import re
import requests
import zipfile
from io import BytesIO
import yaml
import sempy.fabric as fabric

## Download of source & config files
This part downloads all source and config files of FUAM needed for the deployment into the ressources of the notebook

In [ ]:
def download_folder_as_zip(repo_owner, repo_name, output_zip, branch="main", folder_to_extract="src",  remove_folder_prefix = ""):
    # Construct the URL for the GitHub API to download the repository as a zip file
    url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/zipball/{branch}"
    
    # Make a request to the GitHub API
    response = requests.get(url)
    response.raise_for_status()
    
    # Ensure the directory for the output zip file exists
    os.makedirs(os.path.dirname(output_zip), exist_ok=True)
    
    # Create a zip file in memory
    with zipfile.ZipFile(BytesIO(response.content)) as zipf:
        with zipfile.ZipFile(output_zip, 'w') as output_zipf:
            for file_info in zipf.infolist():
                parts = file_info.filename.split('/')
                if  re.sub(r'^.*?/', '/', file_info.filename).startswith(folder_to_extract): 
                    # Extract only the specified folder
                    file_data = zipf.read(file_info.filename)                    
                    output_zipf.writestr(('/'.join(parts[1:]).replace(remove_folder_prefix, "")), file_data)

def uncompress_zip_to_folder(zip_path, extract_to):
    # Ensure the directory for extraction exists
    os.makedirs(extract_to, exist_ok=True)
    
    # Uncompress all files from the zip into the specified folder
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    
    # Delete the original zip file
    os.remove(zip_path)


download_folder_as_zip(repo_owner, repo_name, output_zip = "./builtin/src/src.zip", branch = branch, folder_to_extract= f"{folder_prefix}/src", remove_folder_prefix = f"{folder_prefix}")
download_folder_as_zip(repo_owner, repo_name, output_zip = "./builtin/config/config.zip", branch = branch, folder_to_extract= f"{folder_prefix}/config" , remove_folder_prefix = f"{folder_prefix}")
uncompress_zip_to_folder(zip_path = "./builtin/config/config.zip", extract_to= "./builtin")

## Definition of deployment functions

In [ ]:
def run_fab_command( command, capture_output: bool = False, silently_continue: bool = False):
    result = subprocess.run(["fab", "-c", command], capture_output=capture_output, text=True)
    if (not(silently_continue) and (result.returncode > 0 or result.stderr)):
       raise Exception(f"Error running fab command. exit_code: '{result.returncode}'; stderr: '{result}'")    
    if (capture_output): 
        output = result.stdout.strip()
        return output

def fab_get_id(name):
    id = run_fab_command(f"get /{trg_workspace_name}.Workspace/{name} -q id" , capture_output = True, silently_continue= True)
    return(id)

def fab_get_item(name):
    item = run_fab_command(f"get /{trg_workspace_name}.Workspace/{name}" , capture_output = True, silently_continue= True)
    return(item)

def fab_get_display_name(name):
    display_name = run_fab_command(f"get /{trg_workspace_name}.Workspace/{name} -q displayName" , capture_output = True, silently_continue= True)
    return(display_name)

def fab_get_kusto_query_uri(name):
    connection = run_fab_command(f"get /{trg_workspace_name}.Workspace/{name} -q properties.queryServiceUri", capture_output = True, silently_continue= True)
    return(connection)

def fab_get_kusto_ingest_uri(name):
    connection = run_fab_command(f"get /{trg_workspace_name}.Workspace/{name} -q properties.ingestionServiceUri", capture_output = True, silently_continue= True)
    return(connection)

def fab_get_folders():
    response = run_fab_command(f"api workspaces/{trg_workspace_id}/folders", capture_output = True, silently_continue= True)
    return(json.loads(response).get('text',{}).get('value',[]))

def fab_get_folder(folder_name):
    for f in fab_get_folders():
        if f.get('displayName') == folder_name:
            return f
    return None

def fab_assign_item_folder(name,folder):
    folder_details = fab_get_folder(folder)
    item_id = fab_get_id(name)

    if folder_details is None:
        payload = json.dumps({"displayName": folder})
        folder_details = run_fab_command(f"api -X post workspaces/{trg_workspace_id}/folders -i {payload}", capture_output = True, silently_continue= True)
        folder_details = json.loads(folder_details).get('text',{})

    payload = json.dumps({"folder": folder_details.get('id')})

    return run_fab_command(f"api -X patch workspaces/{trg_workspace_id}/items/{item_id} -i {payload}", capture_output = True, silently_continue= True)

def get_id_by_name(name):
    for it in deployment_order:
        if it.get("name") == name:
                return it.get("id")
    return None

def get_schedule_by_name(name):
    for it in deployment_order:
        if it.get("name") == name:
                return it.get("schedule")
    return None

def add_schedule(name):
    item = run_fab_command(f"get /{trg_workspace_name}.Workspace/{name} -q schedules" , capture_output = True, silently_continue= True)

    if len(json.loads(item)) == 0:
        schedule = get_schedule_by_name(name)

        return run_fab_command(f"job run-sch /{trg_workspace_name}.Workspace/{name} -i {json.dumps(schedule)}" , capture_output = True, silently_continue=True)

    return f"""Job schedule for '{name}' already exists...
* Job schedule {item}""" 

def copy_to_tmp(name):
    shutil.rmtree("./builtin/tmp",  ignore_errors=True)
    path2zip = "./builtin/src/src.zip"
    with  ZipFile(path2zip) as archive:
        for file in archive.namelist():
            if file.startswith(f'src/{name}/'):
                archive.extract(file, './builtin/tmp')
    return(f"./builtin/tmp/src/{name}" )

def replace_ids_in_folder(folder_path, mapping_table):
    for root, _, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith(('.py', '.json', '.pbir', '.platform', '.ipynb', '.tmdl')) and not file_name.endswith('report.json'):
                file_path = os.path.join(root, file_name)
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    for mapping in mapping_table:  
                        content = content.replace(mapping["old_id"], mapping["new_id"])
                with open(file_path, 'w', encoding='utf-8') as file:
                    file.write(content)


## CLI SPN Login

In [ ]:
# Set environment parameters for Fabric CLI
token = notebookutils.credentials.getToken('pbi')
client_id = notebookutils.credentials.getSecret(key_vault_uri, key_vault_client_id)
client_secret = notebookutils.credentials.getSecret(key_vault_uri, key_vault_client_secret)
tenant_id = notebookutils.credentials.getSecret(key_vault_uri, key_vault_tenant_id)

os.environ['FAB_TOKEN_AZURE'] = token

run_fab_command(f"config set encryption_fallback_enabled true" , capture_output = True, silently_continue= True)

run_fab_command(f"auth login -u {client_id} -p {client_secret} --tenant {tenant_id}" , capture_output = False, silently_continue= True)

## Get current Workspace
This cell gets the current workspace to deploy FUAM automatically inside it

In [ ]:
base_path = './builtin/'
config_path = os.path.join(base_path, 'config/item_config.yaml')

with open(config_path, 'r') as file:
        config = yaml.safe_load(file)

deploy_order_path = os.path.join(base_path, 'config/deployment_order.json')
with open(deploy_order_path, 'r') as file:
        deployment_order = json.load(file)

src_workspace_name = config.get('workspace',{}).get('src')
src_capacity_name = config.get('capacity',{}).get('name')

semantic_model_connect_to_lakehouse = config.get('fuam_lakehouse_semantic_models',{})

mapping_table=[]

trg_workspace_id = fabric.get_notebook_workspace_id()
res = run_fab_command(f"api -X get workspaces/{trg_workspace_id}" , capture_output = True, silently_continue=True)
trg_workspace_name = json.loads(res)["text"]["displayName"]

print(f"Current workspace: {trg_workspace_name}")
print(f"Current workspace ID: {trg_workspace_id}")


mapping_table.append({ "old_id": get_id_by_name(src_capacity_name), "new_id": capacity_id })
mapping_table.append({ "old_id": get_id_by_name(src_workspace_name), "new_id": trg_workspace_id })
mapping_table.append({ "old_id": "00000000-0000-0000-0000-000000000000", "new_id": trg_workspace_id })
mapping_table.append({ "old_id": "{key_vault_uri}", "new_id": key_vault_uri })
mapping_table.append({ "old_id": "{key_vault_tenant_id}", "new_id": key_vault_tenant_id })
mapping_table.append({ "old_id": "{key_vault_client_id}", "new_id": key_vault_client_id })
mapping_table.append({ "old_id": "{key_vault_client_secret}", "new_id": key_vault_client_secret })

display(mapping_table)

## Add Group as contributor in the workspace

In [ ]:
if FIRST_RUN:
    payload = json.dumps({
                "principal": {
                    "id": group_id,
                    "type": "Group",
                    "groupDetails": {
                        "groupType": "SecurityGroup",
                    }
                },
                "role": "Admin",
            })

    result = run_fab_command(f"api -X post workspaces/{trg_workspace_id}/roleAssignments -i {payload}" , capture_output = True, silently_continue= True)

## Deployment Logic
This part iterates through all the items, gets the respective source code, replaces all IDs dynamically and deploys the new item

In [ ]:
exclude = [src_workspace_name,src_capacity_name]

for it in deployment_order:

    new_id = None
    
    name = it["name"]

    if name in exclude:
        continue

    if not FIRST_RUN and ".Eventstream" in name:
        continue

    print("")
    print("#############################################")
    print(f"Deploying {name}")

    # Copy and replace IDs in the item
    tmp_path = copy_to_tmp(name)

    cli_parameter = ''
    if ".Notebook" in name:
        cli_parameter = cli_parameter + " --format .py"

    replace_ids_in_folder(tmp_path, mapping_table)    
    
    run_fab_command(f"import  /{trg_workspace_name}.Workspace/{name} -i {tmp_path} -f {cli_parameter} ", silently_continue= True)
    new_id= fab_get_id(name)

    if ".KQLDatabase" in name:
        display_name = fab_get_display_name(name)
        mapping_table.append({ "old_id": "{kusto_db_name}", "new_id": display_name })
    elif ".Eventhouse" in name:
        query_uri = fab_get_kusto_query_uri(name)
        mapping_table.append({ "old_id": "{kusto_query_uri}", "new_id": query_uri })
        ingest_uri = fab_get_kusto_ingest_uri(name)
        mapping_table.append({ "old_id": "{kusto_ingest_uri}", "new_id": ingest_uri })
    mapping_table.append({ "old_id": it["id"], "new_id": new_id })



## Logout Service Principal
Currently the creation of job schedules doesn´t support SPN, so we need to do it with the user

In [ ]:
run_fab_command(f"auth logout", capture_output = True, silently_continue=False)
run_fab_command(f"config set encryption_fallback_enabled false" , capture_output = True, silently_continue= True)

token = notebookutils.credentials.getToken('pbi')
os.environ['FAB_TOKEN'] = token
os.environ['FAB_TOKEN_ONELAKE'] = token


## Create Notebooks Schedules

In [ ]:
for it in deployment_order:
    name = it["name"]
    if ".Notebook" in name:
        schedule = add_schedule(name)

        print(schedule)